In [1]:
import re
from collections import defaultdict

from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
 
from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
 
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [2]:
books = ["data/Rowling, J.K. - HP 1 - Harry Potter and the Sorcerer's Stone.txt",
         "data/Rowling, J.K. - HP 2 - Harry Potter and the Chamber of Secrets.txt",
         "data/Rowling, J.K. - HP 3 - Harry Potter and the Prisoner of Azkaban.txt",
         "data/Rowling, J.K. - HP 4 - Harry Potter and the Goblet of Fire.txt",
         "data/Rowling, J.K. - HP 5 - Harry Potter and the Order of the Phoenix.txt",
         "data/Rowling, J.K. - HP 6 - Harry Potter and the Half-Blood Prince.txt",
         "data/Rowling, J.K. - HP 7 - Harry Potter and the Deathly Hallows.txt"]

In [3]:
pattern = ("((?:[A-Z-][ ]){9,}[A-Z])\s+" +           # Group 1 selects the chapter number
           "([A-Z \n',.-]+)\\b(?![A-Z]+(?=\.)\\b)" + # Group 2 selects the chapter title but excludes edgs of all caps word beginning first sentence of the chapter
           "(?![a-z']|[A-Z.])" +                     # chapter title ends before lowercase letters or a period
           "(.*?)" +                                 # Group 3 selects the chapter contents
           "(?=(?:[A-Z][ ]){9,}|This book \n)")      # chapter contents ends with a new chapter or the end of book
hp = defaultdict(dict)
for book in books:
    title = book[28:-4]
    with open(book, 'r') as f:
        text = f.read()
    chapters = re.findall(pattern, text, re.DOTALL)
    chap = 0
    for chapter in chapters:
        chap += 1
        chap_title = chapter[1].replace('\n','')
        chap_text = (chapter[2][3:].replace('&rsquo;', "'")
                                   .replace('&lsquo;', "'")
                                   .replace('&rdquo;', '"')
                                   .replace('&ldquo;', '"')
                                   .replace('&mdash;', '—'))
        chap_text = re.sub('\n*&bull; [0-9]+ &bull; \n*' + chap_title + ' \n*', '', chap_text, flags=re.IGNORECASE)
        chap_text = re.sub('\n*&bull; [0-9]+ &bull; \s*CHAPTER [A-Z]+ \s*', '', chap_text)
        chap_text = re.sub(' \n&bull; [0-9]+ &bull; \n*', '', chap_text)
#         chap_text = re.sub('\n+', '\n', chap_text)
        chap_text = re.sub('\s*'.join([word for word in chap_title.split()]), '', chap_text)
        hp[title]['Chapter ' + str(chap)] = (chap_title, chap_text)
hp = dict(hp)

### The format of the Harry Potter hp dictionary is as follows:
#### The first book is 'hp1', second is 'hp2', etc.
&nbsp;   
{'hp1': {  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 'Chapter 1': (chapter title, chapter text),  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 'Chapter 2': (chapter title, chapter text),  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 'Chapter 3': (chapter title, chapter text),  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ...  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; }  
 'hp2': {  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 'Chapter 1': (chapter title, chapter text),  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ...  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; }  
 &nbsp;...  
}  

In [4]:
for book in hp:
    print(book)

Harry Potter and the Sorcerer's Stone
Harry Potter and the Chamber of Secrets
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Goblet of Fire
Harry Potter and the Order of the Phoenix
Harry Potter and the Half-Blood Prince
Harry Potter and the Deathly Hallows


In [5]:
language = "english"
sentence_count = 5
 
parser = PlaintextParser(hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][1], Tokenizer(language))

In [6]:
from sumy.summarizers.lex_rank import LexRankSummarizer 
summarizer = LexRankSummarizer(Stemmer(language))
summarizer.stop_words = get_stop_words(language)
#Summarize the document with 2 sentences
summary = summarizer(parser.document, sentence_count) 
for sentence in summary:
    print(sentence)

"The Potters, that's right, that's what I heard —" "— yes, their son, Harry —" Mr. Dursley stopped dead.
Twelve times he clicked the Put-Outer, until the only lights left on the whole street were two tiny pinpricks in the distance, which were the eyes of the cat watching him.
Dumbledore slipped the Put-Outer back inside his cloak and set off down the street toward number four, where he sat down on the wall next to the cat.
"But I c-c-can't stand it — Lily an' James dead — an' poor little Harry off ter live with Muggles —" "Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or we'll be found," Professor McGonagall whispered, patting Hagrid gingerly on the arm as Dumbledore stepped over the low garden wall and walked to the front door.
Dumbledore turned and walked back down the street.


In [7]:
from sumy.summarizers.luhn import LuhnSummarizer
summarizer_1 = LuhnSummarizer(Stemmer(language))
summarizer_1.stop_words = get_stop_words(language)
summary_1 = summarizer_1(parser.document, sentence_count)
for sentence in summary_1:
    print(sentence)

It was now reading the sign that said Privet Drive — no, looking at the sign; cats couldn't read maps or signs.
He didn't see the owls swoop- ing past in broad daylight, though people down in the street did; they pointed and gazed open-mouthed as owl after owl sped overhead.
No one knows why, or how, but they're saying that when he couldn't kill Harry Potter, Voldemort's power somehow broke — and that's why he's gone."
"But I c-c-can't stand it — Lily an' James dead — an' poor little Harry off ter live with Muggles —" "Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or we'll be found," Professor McGonagall whispered, patting Hagrid gingerly on the arm as Dumbledore stepped over the low garden wall and walked to the front door.
G'night, Professor McGonagall — Professor Dumbledore, sir."


In [8]:
from sumy.summarizers.lsa import LsaSummarizer
summarizer_2 = LsaSummarizer(Stemmer(language))
summarizer_2.stop_words = get_stop_words(language)
summary_2 = summarizer_2(parser.document, sentence_count)
for sentence in summary_2:
    print(sentence)

He dashed back across the road, hurried up to his office, snapped at his secretary not to disturb him, seized his telephone, and had almost finished dialing his home number when he changed his mind.
It seemed that Professor McGonagall had reached the point she was most anxious to discuss, the real reason she had been waiting on a cold, hard wall all day, for neither as a cat nor as a woman had she fixed Dumbledore with such a piercing stare as she did now.
He looked simply too big to be allowed, and so wild — long tangles of bushy black hair and beard hid most of his face, he had hands the size of trash can lids, and his feet in their leather boots were like baby dolphins.
For a full minute the three of them stood and looked at the little bundle; Hagrid's shoulders shook, Professor McGonagall blinked furiously, and the twinkling light that usually shone from Dumbledore's eyes seemed to have gone out.
A breeze ruffled the neat hedges of Privet Drive, which lay silent and tidy under the 

In [9]:
from sumy.summarizers.text_rank import TextRankSummarizer
summarizer_3 = TextRankSummarizer(Stemmer(language))
summarizer_3.stop_words = get_stop_words(language)
summary_3 = summarizer_3(parser.document, sentence_count)
for sentence in summary_3:
    print(sentence)

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.
They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made drills.
He was a big, beefy man with hardly any neck, although he did have a very large mustache.
Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors.


/anaconda3/lib/python3.6/site-packages/sumy/summarizers/text_rank.py:68: RuntimeWarning: invalid value encountered in true_divide
  weights /= weights.sum(axis=1)[:, numpy.newaxis]


In [10]:
from sumy.summarizers.edmundson import EdmundsonSummarizer
summarizer_4 = EdmundsonSummarizer(Stemmer(language))
summarizer_4.stop_words = get_stop_words(language)
summarizer_4.bonus_words = hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][0].split()
summarizer_4.stigma_words = ['zdfgthdvndadv']
summarizer_4.null_words = ['zdfgthdvndadv']
summary_4 = summarizer_4(parser.document, sentence_count)
for sentence in summary_4:
    print(sentence)

The Dursleys shuddered to think what the neighbors would say if the Potters arrived in the street.
When Dudley had been put to bed, he went into the living room in time to catch the last report on the evening news: "And finally, bird-watchers everywhere have reported that the nation's owls have been behaving very unusually today.
Twelve times he clicked the Put-Outer, until the only lights left on the whole street were two tiny pinpricks in the distance, which were the eyes of the cat watching him.
Dumbledore slipped the Put-Outer back inside his cloak and set off down the street toward number four, where he sat down on the wall next to the cat.
He couldn't know that at this very moment, people meeting in secret all over the country were holding up their glasses and saying in hushed voices: "To Harry Potter — the boy who lived!"


In [11]:
from sumy.summarizers.edmundson import EdmundsonSummarizer
summarizer_4 = EdmundsonSummarizer(Stemmer(language))
summarizer_4.stop_words = get_stop_words(language)
summarizer_4.bonus_words = hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][0].split()
summarizer_4.stigma_words = ['zdfgthdvndadv']
summarizer_4.null_words = stop_words
summary_4 = summarizer_4(parser.document, sentence_count)
for sentence in summary_4:
    print(sentence)

The Dursleys shuddered to think what the neighbors would say if the Potters arrived in the street.
When Dudley had been put to bed, he went into the living room in time to catch the last report on the evening news: "And finally, bird-watchers everywhere have reported that the nation's owls have been behaving very unusually today.
Twelve times he clicked the Put-Outer, until the only lights left on the whole street were two tiny pinpricks in the distance, which were the eyes of the cat watching him.
Dumbledore slipped the Put-Outer back inside his cloak and set off down the street toward number four, where he sat down on the wall next to the cat.
He couldn't know that at this very moment, people meeting in secret all over the country were holding up their glasses and saying in hushed voices: "To Harry Potter — the boy who lived!"


In [12]:
from sumy.summarizers.sum_basic import SumBasicSummarizer
summarizer_5 = SumBasicSummarizer(Stemmer(language))
summarizer_5.stop_words = get_stop_words(language)
summary_5 = summarizer_5(parser.document, 5)
for sentence in summary_5:
    print(sentence)

Mr. Dursley wondered.
"Harry.
The cat was still there.
"It certainly seems so," said Dumbledore.
"Yes," said Professor McGonagall.


In [13]:
from sumy.summarizers.kl import KLSummarizer
summarizer_6 = KLSummarizer(Stemmer(language))
summarizer_6.stop_words = get_stop_words(language)
summary_6 = summarizer_6(parser.document, sentence_count)
for sentence in summary_6:
    print(sentence)

It was on the corner of the street that he noticed the first sign of something peculiar — a cat reading a map.
It grew steadily louder as they looked up and down the street for some sign of a headlight; it swelled to a roar as they both looked up at the sky — and a huge motorcycle fell out of the air and landed on the road in front of them.
He looked simply too big to be allowed, and so wild — long tangles of bushy black hair and beard hid most of his face, he had hands the size of trash can lids, and his feet in their leather boots were like baby dolphins.
"But I c-c-can't stand it — Lily an' James dead — an' poor little Harry off ter live with Muggles —" "Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or we'll be found," Professor McGonagall whispered, patting Hagrid gingerly on the arm as Dumbledore stepped over the low garden wall and walked to the front door.
He clicked it once, and twelve balls of light sped back to their street lamps so that Privet Drive glowed

In [14]:
from sumy.summarizers.reduction import ReductionSummarizer
summarizer_7 = ReductionSummarizer(Stemmer(language))
summarizer_7.stop_words = get_stop_words(language)
summary_7 = summarizer_7(parser.document, sentence_count)
for sentence in summary_7:
    print(sentence)

Mrs. Potter was Mrs. Dursley's sister, but they hadn't met for several years; in fact, Mrs. Dursley pretended she didn't have a sister, because her sister and her good-for-nothing husband were as unDursleyish as it was possible to be.
It seemed that Professor McGonagall had reached the point she was most anxious to discuss, the real reason she had been waiting on a cold, hard wall all day, for neither as a cat nor as a woman had she fixed Dumbledore with such a piercing stare as she did now.
Dumbledore took Harry in his arms and turned toward the Dursleys' house.
"But I c-c-can't stand it — Lily an' James dead — an' poor little Harry off ter live with Muggles —" "Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or we'll be found," Professor McGonagall whispered, patting Hagrid gingerly on the arm as Dumbledore stepped over the low garden wall and walked to the front door.
G'night, Professor McGonagall — Professor Dumbledore, sir."


In [15]:
from sumy.summarizers.lex_rank import LexRankSummarizer 
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.reduction import ReductionSummarizer

In [16]:
def summarize(text, summarizer, sentence_count, bonus_words=['harry', 'potter'], language='english'):
    summarizer = summarizer(Stemmer(language))
    summarizer.stop_words = get_stop_words(language)
    if isinstance(summarizer, EdmundsonSummarizer):
        summarizer.bonus_words = bonus_words
        summarizer.stigma_words = ['zdfgthdvndadv']
        summarizer.null_words = stop_words
    summary = summarizer(PlaintextParser(text, Tokenizer(language)).document, sentence_count)
    return summary

In [17]:
def print_summary(summary):
    for sentence in summary:
        print(sentence)

In [18]:
for summarizer in [LexRankSummarizer, LuhnSummarizer, LsaSummarizer, TextRankSummarizer,
                   EdmundsonSummarizer, SumBasicSummarizer, KLSummarizer, ReductionSummarizer]:
    print('----' + summarizer.__name__ + '----')
    print_summary(summarize(hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][1],
                            summarizer,
                            sentence_count,
                            bonus_words=hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][0].split()))
    print()

----LexRankSummarizer----
"The Potters, that's right, that's what I heard —" "— yes, their son, Harry —" Mr. Dursley stopped dead.
Twelve times he clicked the Put-Outer, until the only lights left on the whole street were two tiny pinpricks in the distance, which were the eyes of the cat watching him.
Dumbledore slipped the Put-Outer back inside his cloak and set off down the street toward number four, where he sat down on the wall next to the cat.
"But I c-c-can't stand it — Lily an' James dead — an' poor little Harry off ter live with Muggles —" "Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or we'll be found," Professor McGonagall whispered, patting Hagrid gingerly on the arm as Dumbledore stepped over the low garden wall and walked to the front door.
Dumbledore turned and walked back down the street.

----LuhnSummarizer----
It was now reading the sign that said Privet Drive — no, looking at the sign; cats couldn't read maps or signs.
He didn't see the owls swoop-

/anaconda3/lib/python3.6/site-packages/sumy/summarizers/text_rank.py:68: RuntimeWarning: invalid value encountered in true_divide
  weights /= weights.sum(axis=1)[:, numpy.newaxis]


Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.
They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made drills.
He was a big, beefy man with hardly any neck, although he did have a very large mustache.
Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors.

----EdmundsonSummarizer----
The Dursleys shuddered to think what the neighbors would say if the Potters arrived in the street.
When Dudley had been put to bed, he went into the living room in time to catch the last report on the evening news: "And finally, bird-watchers everywhere have reported that the nation's owls have been behaving very unusually today.
Twelve times he cl

In [19]:
# Chapter summaries for all books
for book in hp:
    print(book)
    for chapter in hp[book]:
        print()
        print('{} - {}'.format(chapter, hp[book][chapter][0]))
        print_summary(summarize(hp[book][chapter][1],
                                EdmundsonSummarizer,
                                2,
                                bonus_words=hp[book][chapter][0].split()))
    print()
    print()

Harry Potter and the Sorcerer's Stone

Chapter 1 - THE BOY WHO LIVED
Twelve times he clicked the Put-Outer, until the only lights left on the whole street were two tiny pinpricks in the distance, which were the eyes of the cat watching him.
He couldn't know that at this very moment, people meeting in secret all over the country were holding up their glasses and saying in hushed voices: "To Harry Potter — the boy who lived!"

Chapter 2 - THE VANISHING GLASS
The sun rose on the same tidy front gardens and lit up the brass number four on the Dursleys' front door; it crept into their living room, which was almost exactly the same as it had been on the night when Mr. Dursley had seen that fateful news report about the owls.
Half an hour later, Harry, who couldn't believe his luck, was sitting in the back of the Dursleys' car with Piers and Dudley, on the way to the zoo for the first time in his life.

Chapter 3 - THE LETTERS FROM NO ONE
Yet here it was, a letter, addressed so plainly there 

quo;S WARNING
He saw Dobby's face in the crowd and shouted out, asking for help, but Dobby called, "Harry Potter is safe there, sir!"

Chapter 3 - THE BURROW
The car revved louder and louder and suddenly, with a crunching noise, the bars were pulled clean out of the window as Fred drove straight up in the air.
As long as he wasn't intending to fly the car, the fact that the car could fly wouldn't —" "Arthur Weasley, you made sure there was a loophole when you wrote that law!"

Chapter 4 - AT FLOURISH AND BLOTTS
second-year students will require: The Standard Book of Spells, Grade 2 by Miranda Goshawk Break with a Banshee by Gilderoy Lockhart Gadding with Ghouls by Gilderoy Lockhart Holidays with Hags by Gilderoy Lockhart Travels with Trolls by Gilderoy Lockhart Voyages with Vampires by Gilderoy Lockhart Wanderings with Werewolves by Gilderoy Lockhart Year with the Yeti by Gilderoy Lockhart Fred, who had finished his own list, peered over at Harry's.
Gilderoy Lockhart came slowly into v

Aunt Marge and Aunt Petunia kissed, or rather, Aunt Marge bumped her large jaw against Aunt Petunia's bony cheekbone.
yelled Uncle Vernon and Aunt Petunia together as Aunt Marge's whole body began to rise off her chair toward the ceiling.

Chapter 3 - THE KNIGHT BUS
There was another tremendous BANG, and the next moment Harry found himself flat on his bed, thrown backward by the speed of the Knight Bus.
"Anyway, they cornered Black in the middle of a street full of Muggles an' Black took out 'is wand and 'e blasted 'alf the street apart, an' a wizard got it, an' so did a dozen Muggles what got in the way.

Chapter 4 - THE LEAKY CAULDRON
Harry ate breakfast each morning in the Leaky Cauldron, where he liked watching the other guests: funny little witches from the country, up for a day's shopping; venerable-looking wizards arguing over the latest article in Transfiguration Today; wild-looking warlocks; raucous dwarfs; and once, what looked suspiciously like a hag, who ordered a plate of 

quo;S SECRET
On the other hand, their interference might have permitted HERMIONE'S SECRET


Harry Potter and the Goblet of Fire

Chapter 1 - THE RIDDLE HOUSE
But over in the neighboring town of Great Hangleton, in the dark and dingy police station, Frank was stubbornly repeating, again and again, that he was innocent, and that the only person he had seen near the house on the day of the Riddles' deaths had been a teenage boy, a stranger, dark-haired and pale.
The only means of escape was into the room where two men sat plotting murder, yet if he stayed where he was the snake would surely kill him — But before he had made his decision, the snake was level with him, and then, incredibly, miraculously, it was passing; it was following the spitting, hissing noises made by the cold voice beyond the door, and in seconds, the tip of its diamond-patterned tail had vanished through the gap.

Chapter 2 - THE SCAR
He sat up, one hand still on his scar, the other reaching out in the darkness for h

Harry heard the buzzing of the hundreds of students on the other side of the wall, before Professor McGonagall closed the door.
He wrenched the hangings shut around his four-poster, leaving Harry standing there by the door, staring at the dark red velvet curtains, now hiding one of the few people he had been sure would believe him.

Chapter 18 - THE WEIGHING OF THE WANDS
The prospect of going down into the Great Hall and facing the rest of the Gryffindors, all treating him like some sort of hero, was not inviting; it was that, however, or stay here and allow himself to be cornered by the Creevey brothers, who were both beckoning frantically to him to join them.
He was in a fairly small classroom; most of the desks had been pushed away to the back of the room, leaving a large space in the middle; three of them, however, had been placed end-to-end in front of the blackboard and covered with a long length of velvet.

Chapter 19 - THE HUNGARIAN HORNTAIL
In the meantime, life became even wo

...

Ron's euphoria at helping Gryffindor scrape the Quidditch Cup was such that he could not settle to anything next day.
"Well, I mean, I'd already let in that one of Davies's, so I wasn't feeling that confident, but I dunno, when Bradley came toward me, justCHAPTER THIRTY-ONE

Chapter 31 - OUT OF THE FIRE I
"Okay," Harry said aggressively to Hermione, "Okay, if you can think of a way of doing this quickly, I'm with you, otherwise I'm going to the Department of Mysteries right now —" "The Department of Mysteries?"
He inhaled a lot of ash and, choking, found himself being dragged backward through the flames until, with a horrible abruptness, he was staring up into the wide, pallid face of Professor Umbridge, who had dragged him backward out of the fire by the hair and was now bending his neck back as far as it would go as though she was going to slit his throat.

Chapter 32 - FIGHT AND FLIGHT
Harry's centaur let him fall to the ground again as his hands flew to his bow and quiver of arrows

Katie was removed to St. Mungo's Hospital for Magical Maladies and Injuries the following day, by which time the news that she had been cursed had spread all over the school, though the details were confused and nobody other than Harry, Ron, Hermione, and Leanne seemed to know that Katie herself had not been the intended target.
Dumbledore handed Riddle the envelope containing his list of equipment, and after telling Riddle exactly how to get to the Leaky Cauldron from the orphanage, he said, "You will be able to see it, although Muggles around you — non-magical people, that is — will not.

Chapter 14 - FELIX FELICIS
Felix Felicis!"
"You added Felix Felicis to Ron's juice this morning, that's why he saved everything!

Chapter 15 - THE UNBREAKABLE VOW
Hagrid had already single-handedly delivered the usual twelve Christmas trees for the Great Hall; garlands of holly and tinsel had been twisted around the banisters of the stairs; everlasting candles glowed from inside the helmets of suits

The sound of the front door slamming echoed up the stairs and a voice yelled, "Oi!
For a moment Harry had the strangest feeling that she wanted to say something to him: She gave him an odd, tremulous look and seemed to teeter on the edge of speech, but then, with a little jerk of her head, she bustled out of the room after her husband and son.

Chapter 4 - THE SEVEN POTTERS
The broomstick spun to earth, but he just managed to seize the strap of his rucksack and the top of the cage as the motorbike swung the right way up again.
The jinx hit the middle Death Eater in the chest: For a moment the man was absurdly spread-eagled in midair as though he had hit an invisible barrier: One of his fellows almost collided with him — Then the sidecar began to fall in earnest, and the remaining Death Eater shot a curse so close to Harry that he had to duck below the rim of the car, knocking out a tooth on the edge of his seat — "I'm comin', Harry, I'm comin'!"

Chapter 5 - FALLEN WARRIOR
Hagrid?"
Sta

"It's just a morality tale, it's obvious which gift is best, which one you'd choose —" The three of them spoke at the same time; Hermione said, "the Cloak," Ron said, "the wand," and Harry said, "the stone."
The bust of Rowena Ravenclaw lay beside him with half its face missing, fragments of torn parchment were floating through the air, and most of the printing press lay on its side, blocking the top of the staircase to the kitchen.

Chapter 22 - THE DEATHLY HALLOWS
His Cloak was the third Hallow, and when he discovered how to open the Snitch he would have the second, and then all he needed to do was find the first Hallow, the Elder Wand, and then — But it was as though a curtain fell on a lit stage: All his excitement, all his hope and happiness were extinguished at a stroke, and he stood alone in the darkness, and the glorious spell was broken.
The idea of the Deathly Hallows had taken possession of him, and he could not rest while agitating thoughts whirled through his mind: the wan

How many would there be time for, as he rose and walked through the castle for the last time, out into the grounds and into the forest?CHAPTER THIRTY-FOUR
The dead who walked beside him through the forest were much more real to him now than the living back at the castle: Ron, Hermione, Ginny, and all the others were the ones who felt like ghosts as he stumbled and slipped toward the end of his life, toward Voldemort.

Chapter 35 - KING
quo;S CROSS
"Tell me one last thing," said Harry.




In [20]:
chapter_titles = []
for book in hp:
    chapter_titles.append(' '.join([hp[book][chapter][0] for chapter in hp[book]]))
chapter_titles = [title.split() for title in chapter_titles]
# or
chapter_titles = [title.split() for title in [' '.join([hp[book][chapter][0] for chapter in hp[book]]) for book in hp]]

In [21]:
book_text = [(' '.join([hp[book][chapter][1] for chapter in hp[book]])) for book in hp]

In [22]:
# Book summaries for all books
for book in hp:
    print(book)
    print()
    print_summary(summarize((' '.join([hp[book][chapter][1] for chapter in hp[book]])),
                            EdmundsonSummarizer,
                            5,
                            bonus_words=[title.split()
                                         for title in [' '.join([hp[book][chapter][0]
                                                                 for chapter in hp[book]])
                                                       for book in hp]]))
    print()
    print()

Harry Potter and the Sorcerer's Stone

The guard had never heard of Hogwarts and when Harry couldn't even tell him what part of the country it was in, he started to get annoyed, as though Harry was being stupid on purpose.
Harry could hear the drone of hundreds of voices from a doorway to the right — the rest of the school must already be here — but Professor McGonagall showed the first years into a small, empty chamber off the hall.
And the school bellowed: "Hogwarts, Hogwarts, Hoggy Warty Hogwarts, Teach us something please, Whether we be old and bald Or young with scabby knees, Our heads could do with filling With some interesting stuff, For now they're bare and full of air, Dead flies and bits of fluff, So teach us things worth knowing, Bring back what we've forgot, Just do your best, we'll do the rest, And learn until our brains all rot."
And then the story started to spread: Harry Potter, the famous Harry Potter, their hero of two Quidditch matches, had lost them all those points

One tangled itself in Hermione's hair, and Ron beat it back with a pair of secateurs; Harry succeeded in trapping a couple of vines and knotting them together; a hole opened in the middle of all the tentaclelike branches; Hermione plunged her arm bravely into this hole, which closed like a trap around her elbow; Harry and Ron tugged and wrenched at the vines, forcing the hole to open again, and Hermione snatched her arm free, clutching in her fingers a pod just like Neville's.
He was not fooled; for all Scrimgeour's talk that they had just been in the area, that Percy wanted to look up his family, this must be the real reason that they had come, so that Scrimgeour could speak to Harry alone.
McLaggen kept up a constant stream of hints that he would make a better permanent Keeper for the team than Ron, and that now that Harry was seeing him play regularly he would surely come around to this way of thinking too; he was also keen to criticize the other players and provide Harry with detai